# Programming Assignment #1

In [1]:
import sys
sys.path.append('.')
sys.path.append('..')
from collections import defaultdict, namedtuple
from edge_pairs import preprocess_directed_edge_pairs as preprocessor
from problem_loader import ProblemLoader
import sys, threading
sys.setrecursionlimit(800000)
threading.stack_size(67108864)


data_urls = {
    'problem1': 'https://d18ky98rnyall9.cloudfront.net/_410e934e6553ac56409b2cb7096a44aa_SCC.txt?Expires=1622592000&Signature=XSp6f8VpCzozJTkVMqbkaYlNxTP41GlG2NtviHXJuxnbBCA78Tpa5tkEo0XnzIIHGcd8MQVY1nE8EgsjMFPIl2fl7WTW9VCUhOrMbDLcEuX0BY170iJSVfus9-ESG0dWtyZgzOdOqXPzdGj88EWWRdlW2An4SiH2uUdFNNpFKt8_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A'
}

## Problem 1

### Description
The file contains the edges of a directed graph. Vertices are labeled as positive integers from 1 to 875714. Every row indicates an edge, the vertex label in first column is the tail and the vertex label in second column is the head (recall the graph is directed, and the edges are directed from the first column vertex to the second column vertex). So for example, the $11^{th}$ row looks liks: `2 47646`. This just means that the vertex with label _2_ has an outgoing edge to the vertex with label _47646_

Your task is to code up the algorithm from the video lectures for computing strongly connected components (SCCs), and to run this algorithm on the given graph.

### Output Format: 
You should output the sizes of the 5 largest SCCs in the given graph, in decreasing order of sizes, separated by commas (avoid any spaces). So if your algorithm computes the sizes of the five largest SCCs to be 500, 400, 300, 200 and 100, then your answer should be "500,400,300,200,100" (without the quotes). If your algorithm finds less than 5 SCCs, then write 0 for the remaining terms. Thus, if your algorithm computes only 3 SCCs whose sizes are 400, 300, and 100, then your answer should be "400,300,100,0,0" (without the quotes).  (Note also that your answer should not have any spaces in it.)

### WARNING: 
This is the most challenging programming assignment of the course. Because of the size of the graph you may have to manage memory carefully. The best way to do this depends on your programming language and environment, and we strongly suggest that you exchange tips for doing this on the discussion forums.



In [2]:
values = ProblemLoader(
    data_urls['problem1'], 
    fname="graph.p", 
    preprocessor=preprocessor
).fetch()
print(values[:2])

[(1, 1), (1, 2)]


In [3]:
Node = namedtuple('Node', ['children', 'parents'])

class Graph():
    def __init__(self):
        self.nodes = defaultdict(list)

    def add_edge(self, x, y):
        if x in self.nodes:
            self.nodes[x].children.append(y)
        else:
            self.nodes[x] = Node(children=[y], parents=[])
        if y in self.nodes:
            self.nodes[y].parents.append(x)
        else:
            self.nodes[y] = Node(children=[], parents=[x])
    
    def dfs(self, i, visited_vertex, stack, reversed=False):
        visited_vertex[i] = True

        branch = 'parents' if reversed else 'children'
        for j in getattr(self.nodes[i], branch):
            if not visited_vertex[j]:
                self.dfs(j, visited_vertex, stack, reversed)

        stack = stack.append(i)
        

In [4]:
g = Graph()
for x, y in values:
    g.add_edge(x,y)


In [5]:
def main():
    stack = [] # the ordering for the second pass
    visited_vertex = { k: False for k in g.nodes.keys() }
    for i in range(len(g.nodes.keys()), 0, -1):
        if not visited_vertex[i]:
            g.dfs(i, visited_vertex, stack, reversed=True)

    sccs=[]
    visited_vertex = { k: False for k in g.nodes.keys() }
    while stack:
        scc=[]
        i = stack.pop()
        if not visited_vertex[i]:
            g.dfs(i, visited_vertex, scc)
        sccs.append(scc)

    lengths = list(map(len, sccs))
    lengths.sort(reverse=True)
    print(str(lengths[:5]))

thread = threading.Thread(target=main)
thread.start()

[434821, 968, 459, 313, 211]
